### Notes:

The following demonstrates linking to, running from and displaying one notebook from inside another notebook. It could be used in combination with a mindmapping tool to better structure research notebooks.
It makes uses of information shown in these two demo notebooks:
    
Minrk's treatment of notebooks as [a module](http://nbviewer.ipython.org/6011986)
and another [notebook](http://nbviewer.ipython.org/5491090/analysis.ipynb) by an anonymous author doing something similar.

Also more recently note [Fernando's gist](http://nbviewer.ipython.org/gist/fperez/9716279)

### Contents

We start with a link to local notebook [test](http://127.0.0.1:8888/Test_1.ipynb) inside a markdown cell.

In [1]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

from IPython.display import display, HTML
from IPython.nbformat import current
#from IPython.core.interactiveshell import InteractiveShell

import io 

formatter = HtmlFormatter()
lexer = PythonLexer()

# publish the CSS for pygments highlighting
display(HTML("""
<style type='text/css'>
%s
</style>
""" % formatter.get_style_defs()
))

def show_notebook(fname):
    """display a short summary of the cells of a notebook"""
    with io.open(fname, 'r', encoding='utf-8') as f:
        nb = current.read(f, 'json')
    html = []
    for cell in nb.worksheets[0].cells:
        html.append("<h4>%s cell</h4>" % cell.cell_type)
        if cell.cell_type == 'code':
            html.append(highlight(cell.input, lexer, formatter))
        else:
            html.append("<pre>%s</pre>" % cell.source)
    display(HTML('\n'.join(html)))

show_notebook("Inter_Notebook_Test_Dummy -Complete.ipynb")

/home/clyde/anaconda/envs/test3/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)


In [2]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

from IPython.display import display, HTML
from IPython import nbformat

#from IPython.core.interactiveshell import InteractiveShell

import io 

formatter = HtmlFormatter()
lexer = PythonLexer()

# publish the CSS for pygments highlighting
display(HTML("""
<style type='text/css'>
%s
</style>
""" % formatter.get_style_defs()
))

def show_notebook(fname):
    """display a short summary of the cells of a notebook"""
    with io.open(fname, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f,as_version=3)
    html = []
    for cell in nb.worksheets[0].cells:
        html.append("<h4>%s cell</h4>" % cell.cell_type)
        if cell.cell_type == 'code':
            html.append(highlight(cell.input, lexer, formatter))
        else:
            html.append("<pre>%s</pre>" % cell.source)
    display(HTML('\n'.join(html)))

show_notebook("Inter_Notebook_Test_Dummy -Complete.ipynb")

In [3]:
import io
from IPython.nbformat import current

def execute_notebook(nbfile):
    
    with io.open(nbfile) as f:
        nb = nbformat.read(f,as_version=3)
    
    ip = get_ipython()
    
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
    
execute_notebook('Inter_Notebook_Test_Dummy -Complete.ipynb')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


'Cell 3'

'Last cell'

In [4]:
#Testing partial inheritence

import io
import re
from IPython.nbformat import current

def extract_linked_nb(mkdn_cell):
    nb_link_extractor = re.compile("(?<=\]\().+ipynb")
    linked_nbs = nb_link_extractor.findall(mkdn_cell['source'])
    return linked_nbs

def inherit_upto_cell(nb_name, cell_id=-1, silent=False):
    """inherits from another notebook up to cell number cell_id"""

    ip = get_ipython()

    if '.ipynb' not in nb_name:
        nb_name += '.ipynb'

    with io.open(nb_name) as f:
        ancestor_nb = nbformat.read(f,as_version=3)

    code_cells = [cell for cell in ancestor_nb.worksheets[0].cells if cell.cell_type == 'code']

    if cell_id == -1:
        cell_id = len(code_cells)
        print('Inheriting entire notebook')

    ancestor_code_cells = [cell for i,cell in enumerate(code_cells) if i <= cell_id]

    for cell in ancestor_code_cells:
        ip.run_cell(cell.input, silent=silent)

    return

def inherit_from(nb_name, uptolink='', silent=False):
    ip = get_ipython()

    if '.ipynb' not in nb_name:
        nb_name += '.ipynb'

    with io.open(nb_name) as f:
        ancestor_nb = nbformat.read(f,as_version=3)

    cell_id=-1
    for cell in ancestor_nb.worksheets[0].cells:
        if cell.cell_type == 'code':
            cell_id+=1
        if cell.cell_type == 'markdown' and uptolink in extract_linked_nb(cell):
            break
    else:
        inherit_upto_cell(nb_name, cell_id=-1, silent=silent)
        return
    
    inherit_upto_cell(nb_name, cell_id=cell_id, silent=silent)

In [5]:
inherit_upto_cell('Inter_Notebook_Test_Dummy -Complete.ipynb')

Inheriting entire notebook
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


'Cell 3'

'Last cell'

In [6]:
inherit_from('Inter_Notebook_Test_Dummy -Complete.ipynb', uptolink='Inter Notebook Test - Complete.ipynb')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [7]:
nb_name='Inter_Notebook_Test_Dummy -Complete.ipynb'
uptolink='Inter Notebook Test - Complete.ipynb'

ip = get_ipython()

if '.ipynb' not in nb_name:
    nb_name += '.ipynb'

with io.open(nb_name) as f:
    ancestor_nb = nbformat.read(f,as_version=3)

cell_id=-1
for cell in ancestor_nb.worksheets[0].cells:
    if cell.cell_type == 'code':
        cell_id+=1
    if cell.cell_type == 'markdown' and uptolink in extract_linked_nb(cell):
        break

cell_id

1

In [8]:
ip = get_ipython()

#set current_nb_name with this horrible hack
ip.run_code("""from IPython.display import Javascript\nJavascript('IPython.notebook.kernel.execute("current_nb_name = " + "\'"+IPython.notebook.notebook_name+"\'");')""")
code_cell_id = 0

if '.ipynb' not in nb_name:
    nb_name += '.ipynb'

with io.open(nb_name) as f:
    ancestor_nb = nbformat.read(f,as_version=3)

id=0
for cell in ancestor_nb.worksheets[0].cells:
    if cell.cell_type == 'code':
        id+=1
    if cell.cell_type == 'markdown':
        print(extract_linked_nb(cell))
        print(cell['source'])

[]
This is part of a pair of notebooks showing calling of cells from notebook to another.

Below is a print loop inside the Inter_Notebook_Test_Dummy notebook
[u'Inter Notebook Test - Complete.ipynb']
If we wanted to inherit up to this point we would put the link to the inheriting notebook [here](Inter Notebook Test - Complete.ipynb)


In [9]:
ip.run_cell("""import IPython.display\nfrom IPython.display import Javascript\ndisplay(Javascript('IPython.notebook.kernel.execute("current_nb_name = " + "\'"+IPython.notebook.notebook_name+"\'")'))""")

<IPython.core.display.Javascript object>

In [10]:
ip.run_code("current_nb_name='test'")

0

In [11]:
current_nb_name

'test'

In [12]:
mk_cells = [cell for cell in ancestor_nb.worksheets[0].cells if cell.cell_type=='markdown']

In [13]:
mk_cells

[{u'cell_type': u'markdown',
  u'metadata': {},
  u'source': u'This is part of a pair of notebooks showing calling of cells from notebook to another.\n\nBelow is a print loop inside the Inter_Notebook_Test_Dummy notebook'},
 {u'cell_type': u'markdown',
  u'metadata': {},
  u'source': u'If we wanted to inherit up to this point we would put the link to the inheriting notebook [here](Inter Notebook Test - Complete.ipynb)'}]

In [14]:
current_nb_name

'test'

In [15]:
from cc_notebook import inherit_from

inherit_from(nb_name,uptolink=uptolink)

<IPython.core.display.Javascript object>

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [9]:
inherit_from?